# 02.0 - Unity Catalog checks

Let's first inspect where can we write our data:

In [0]:
%sql
SHOW CATALOGS;

In [0]:
%sql
-- Replace with the appropriate catalog name!
SHOW GRANTS ON CATALOG `aza-databricks-b9b7aae-catalog`;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS `aza-databricks-b9b7aae-catalog`.$LASTNAME_rocketfuel;

Now we are ready to start ingesting data into the schema.

Loading data directly from the workspace may be disabled depending on the cluster configuration.

As such, we will create a Volume in Unity Catalog:

In [0]:
%sql
SHOW VOLUMES IN `aza-databricks-b9b7aae-catalog`.rocketfuel

In [0]:
%sql
CREATE VOLUME `aza-databricks-b9b7aae-catalog`.rocketfuel.bronze;

Now either upload `companies.csv`, `reviews.csv` and `shuttles.xlsx` manually using the Unity Catalog UI, or run this code from your local machine:

In [ ]:
from itertools import chain
from pathlib import Path

from databricks.sdk import WorkspaceClient

w = WorkspaceClient()

local_data_folder = Path.cwd() / "rocketfuel" / "data" / "01_raw"

for datafile in chain(local_data_folder.glob("*.csv"), local_data_folder.glob("*.xlsx")):
    volume_path = f"/Volumes/aza-databricks-b9b7aae-catalog/rocketfuel/bronze/{datafile.name}"

    w.files.upload(
        volume_path,
        contents=datafile.read_bytes(),
        overwrite=True,
    )

In [ ]:
%sql
LIST '/Volumes/aza-databricks-b9b7aae-catalog/rocketfuel/bronze/'